In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from pymongo import MongoClient
import utils, os
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:

data = utils.read_data()
embedding = OpenAIEmbeddings(model='text-embedding-3-small')  # 1536-dim


/var/folders/s2/tb48c2wd03jbyvkwy3zmp8gw0000gn/T/ipykernel_35743/3378671910.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model='text-embedding-3-small')  # 1536-dim


In [5]:
client = MongoClient(os.getenv("MONGODB_URI"))
collection = client["RAG-evaluation"]["RAG-simple"]
vectorstore = MongoDBAtlasVectorSearch(
            embedding=embedding,
            collection=collection,
            index_name="vector_index"
        )


/var/folders/s2/tb48c2wd03jbyvkwy3zmp8gw0000gn/T/ipykernel_35743/773328371.py:3: LangChainDeprecationWarning: The class `MongoDBAtlasVectorSearch` was deprecated in LangChain 0.0.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-mongodb package and should be used instead. To use it run `pip install -U :class:`~langchain-mongodb` and import as `from :class:`~langchain_mongodb import MongoDBAtlasVectorSearch``.
  vectorstore = MongoDBAtlasVectorSearch(


In [6]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo")

/opt/anaconda3/lib/python3.11/site-packages/langchain_community/llms/openai.py:255: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/langchain_community/llms/openai.py:1089: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

question = "how to delay skin aging"

prompt_str = f"""You are an AI assistant that generates hypothetical answers ('Hyde').
Given the question '{question}', generate a hypothetical document that directly answers this question.
The document should be detailed and in-depth and exactly 500 characters."""

# ✅ Latest LangChain API
hyde_answer = llm.invoke(prompt_str)



/var/folders/s2/tb48c2wd03jbyvkwy3zmp8gw0000gn/T/ipykernel_35743/368802819.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)


To delay skin aging, start by protecting your skin from the sun with SPF, staying hydrated, and maintaining a healthy diet rich in antioxidants. Use skincare products with retinoids and hyaluronic acid, and incorporate a daily moisturizer. Avoid smoking and excessive alcohol consumption, and prioritize getting enough sleep and managing stress. Regularly exfoliate and use face masks to promote cell turnover. Lastly, consider incorporating collagen supplements or treatments like chemical peels or laser therapy for more advanced anti-aging benefits.


In [21]:
retrieved_docs = retriever.get_relevant_documents(hyde_answer.content)

print("\n--- Retrieved Chunks based on Hyde ---")
for i, doc in enumerate(retrieved_docs):
    print(f"Rank {i+1}: {doc.page_content[:200]}...\n")

/var/folders/s2/tb48c2wd03jbyvkwy3zmp8gw0000gn/T/ipykernel_35743/504867814.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(hyde_answer.content)



--- Retrieved Chunks based on Hyde ---
Rank 1: like ﬁne lines, Dr. Skotnicki recommends starting to use retinol products around age 30 to get
ahead of the game. Retinoids are also notorious for causing irritation when you ﬁrst start using
them, so...

Rank 2: like ﬁne lines, Dr. Skotnicki recommends starting to use retinol products around age 30 to get
ahead of the game. Retinoids are also notorious for causing irritation when you ﬁrst start using
them, so...

Rank 3: aging. ( e other is sunscreen!) Retinoids, which are forms of vitamin A, work by stimulating
the skin-cell-shedding process from below, leading to smoother skin and a reduction in both
signs of aging ...

Rank 4: aging. ( e other is sunscreen!) Retinoids, which are forms of vitamin A, work by stimulating
the skin-cell-shedding process from below, leading to smoother skin and a reduction in both
signs of aging ...

Rank 5: giving your skin some love has both cosmetic and medical beneﬁts. For instance, although you
can’t s